In [ ]:
%load_ext autoreload
%autoreload 2

import hydra

from openretina.data_io.hoefling_2024.stimuli import movies_from_pickle
from openretina.insilico.VectorFieldAnalysis.vector_field_analysis import *
from openretina.models.core_readout import load_core_readout_from_remote
from openretina.utils.file_utils import get_local_file_path

In this notebook, we will introduce how to use OpenRetina to compute the vector field of local Spike Triggered Average as described in Goldin et al. 2022. This analysis can be used as a complement to the Most Discriminative Stimulus analysis The method should work for most models trained in the OpenRetina framework but the user might need to readapt how the dataset of natural images is constructed (usually from snippets of the training dataset).

# Load model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_quality = "low"
model_name = f"hoefling_2024_base_{model_quality}_res"
model = load_core_readout_from_remote(
    model_name, device="cuda" if torch.cuda.is_available() else "cpu"
)
model

In [ ]:
# pick a session if needed
session_id = 'session_3_ventral2_20210910'  # list(model.readout.keys())[0]
print('Using session : ', session_id)
n_neurons = model.readout[session_id].outdims
print(f"Number of neurons: {n_neurons}")

In [ ]:
sessions = list(model.readout.keys())

# Optional / We will reshape the cell type annotations into a new format to use it later down
group_sessions_cells = {}

# Iterate through all sessions
for session in sessions:
    # Get assignments for this session
    assignments = model.data_info["sessions_kwargs"][session]["group_assignment"]

    # Get number of neurons for this session
    n_neurons = model.readout[session].outdims

    # For each cell in this session
    for cell_id in range(n_neurons):
        group = int(assignments[cell_id])  # Convert to int for dictionary key

        # Initialize group if not exists
        if group not in group_sessions_cells:
            group_sessions_cells[group] = {}

        # Initialize session if not exists for this group
        if session not in group_sessions_cells[group]:
            group_sessions_cells[group][session] = []

        # Add cell ID to the appropriate group and session
        group_sessions_cells[group][session].append(cell_id)

# Load natural images

We need to create a dataset of natural images to which we will compute the lSTA to. For movie models we can simply take random frames from the movies. We recommend using ~3000 images.

In [ ]:
# In the case of Hoefling et al, let's load training data
with hydra.initialize(config_path=os.path.join("..", "configs"), version_base="1.3"):
    cfg = hydra.compose(config_name=f"hoefling_2024_core_readout_{model_quality}_res.yaml")

movies_path = get_local_file_path(file_path=cfg.paths.movies_path, cache_folder=cfg.paths.data_dir)

movies_dict = movies_from_pickle(movies_path)

In [ ]:
# We will just take equally intervleaved frames from the training movies to create a dataset of natural images.
n_images = 3000
rate = movies_dict.train.shape[1] // n_images
natural_images_library = movies_dict.train[:, ::rate, :, :].swapaxes(0, 1)

In [ ]:
# The next function will turn our images into singular sequence grey -> frame, using a length of grey depending on the size
# of the model temporal filtering
# The n_image_frames is the number of frames the natural image is displayed, depending on the window you want to average
# the cell response for LSTA computation (see below).
movies, n_empty_frames = prepare_movies_dataset(model, session_id,
    normalize_movies=False, # Already normalized
    image_library=natural_images_library, n_image_frames=60, device = device)

In [ ]:
# Display the last frame of an example movie with both channels side by side
example_idx = 150  # First movie as example
last_frame = movies[example_idx, :, -1] # movies[example_idx, :, -1]  # Shape: (2, 72, 64)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# First channel (channel 0)
im1 = axes[0].imshow(last_frame[0], cmap='Greens', vmin=movies[:, 0].min(), vmax=movies[:, 0].max())
axes[0].set_title('First Channel')
plt.colorbar(im1, ax=axes[0])
axes[0].axis('off')
# Optional : Second channel (channel 1)
im2 = axes[1].imshow(last_frame[1], cmap='Purples', vmin=movies[:, 1].min(), vmax=movies[:, 1].max())
axes[1].set_title('Second Channel')
plt.colorbar(im2, ax=axes[1])
axes[1].axis('off')
plt.tight_layout()
plt.show()

print(f"First channel range: [{last_frame[0].min():.3f}, {last_frame[0].max():.3f}]")
print(f"Second channel range: [{last_frame[1].min():.3f}, {last_frame[1].max():.3f}]")

# Draw the vector field for a selected cell

In [ ]:
cell_id = 11  # TO DO: CHOOSE BETTER EXQMPLE

# The following function will compute the response and the lSTA to all the images in the movies dataset
# Choose an integration window that makes sense for your model (here 5-40 frames after image onset, i.e. 0.17s to 1.33s
#  after image onset)
lsta_library, response_library = compute_lsta_library(
    model,
    movies[:n_images], # Can limit number of images for it to run faster
    session_id,
    cell_id,
    integration_window=(0, 30),  # Integration window for the LSTA
    batch_size=64,
    device=device,
)

In [ ]:
lsta_library.shape, response_library.shape

In [ ]:
# We can check some sample responses, making sure the integrity of the response profile is predicted and we did not
#  cut it too short.
# This view can be used to help select the integration window (typical number of frames on the cell response
#  in the dataset)

plt.plot(response_library[0, :, cell_id])
plt.plot(response_library[1, :, cell_id])
plt.plot(response_library[2, :, cell_id])
plt.plot(response_library[3, :, cell_id])
plt.plot(response_library[4, :, cell_id])
plt.plot(response_library[5, :, cell_id])
plt.xlabel('Time (frames)')
plt.ylabel('Cell predicted Response (au)')

In [ ]:
# We can now plot an example LSTA and the corresponding image

image = 450
channel = 1
lsta = lsta_library[image, channel]
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# image
axes[0].imshow(movies[image,0,-1], cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# LSTA
axes[1].imshow(lsta, cmap='bwr', vmin=-abs(lsta).max(), vmax=abs(lsta).max())
axes[1].set_title('LSTA (Local Spatiotemporal Average)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# PCA on the LSTA library

Interpretation : Here we are drawing a approximation of what are the spatial components of the cell lSTA. They are the two axis directing the cell selectivity, usually the first PC looks like a simple receptive field and can be seen as a 'luminance' axis, while the second PC is more complex spatially and can be seen as a 'local contrast' axis.

In [ ]:
# Select channel if using a color channels model
channel = 0

PC1, PC2, explained_variance = get_pc_from_pca(model, channel, lsta_library, plot=True)

In [ ]:
# Project the images onto PCA space
images = movies[:,channel,-1,:,:]
images_coordinate = get_images_coordinate(images, PC1, PC2, plot=True)
plt.axis('off')
plt.axis('off')

# Plot the vector field

Interpretation :
We can project both the images (represented as arrow basis) and the corresponding lSTAs (represented as arrow heads) on the two first PCs. This give use a view of the geometry of the response space of the cell. If the arrow all points in the same horizontal direction, the cell consistently encode the same thing. If the directions arrows vary, the cell selectivity depends on the current context. Arrows diverging can be interpreted as a preference for increase in local contrast in a part of stimulus space, while arrows convergind could be a preference for decrease in local contrast.

In [ ]:
# OPTIONAL : Plot the raw vector field of the LSTA in PCA space
# fig = plot_untreated_vectorfield(lsta_library, channel, PC1, PC2, images_coordinate)

In [ ]:
# Plot the optimiz vector field of the LSTA in PCA space
fig = plot_clean_vectorfield(
    lsta_library,
    channel,
    PC1,
    PC2,
    images,
    images_coordinate,
    explained_variance,
    x_bins=30,
    y_bins=30,
)
fig.show()

# ADD the firing rate by the side + mention cell types

# Optional : Create all arrowplots of a session

In [ ]:
# for session_id in list(model.readout.keys())[13:]:
#     os.makedirs(f"hoefling_low_res_vector_fields/{session_id}", exist_ok=True)
#     n_neurons = model.readout[session_id].outdims
#     print(n_neurons)
#     movies, n_empty_frames = prepare_movies_dataset(
#         model,
#         session_id,
#         normalize_movies=False,  # Already normalized
#         image_library=natural_images_library,
#         n_image_frames=60,
#         device=device,
#     )

#     for cell_id in range(n_neurons):
#         print(f"Processing cell {cell_id}...")
#         lsta_library, response_library = compute_lsta_library(
#             model, movies, session_id, cell_id, batch_size=64, device=device
#         )
#         assignment = model.data_info['sessions_kwargs'][session_id]['group_assignment'][cell_id]
#         for channel in range(2):
#             PC1, PC2, explained_variance = get_pc_from_pca(model, channel, lsta_library, plot=True)
#             images_coordinate = get_images_coordinate(images, PC1, PC2, plot=False)
#             fig = plot_clean_vectorfield(
#                 lsta_library,
#                 channel,
#                 PC1,
#                 PC2,
#                 images,
#                 images_coordinate,
#                 explained_variance,
#                 x_bins=31,
#                 y_bins=31,
#             )
#             plt.savefig(
#                 f"hoefling_low_res_vector_fields/{session_id}/cell_{cell_id}_channel_{channel}_group_{assignment}.png",
#                 dpi=300,
#                 bbox_inches="tight",
#             )
#             plt.close("all")